In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

folder = os.path.join('example_data', 'final_data')

In [2]:
Zdat = np.load(os.path.join(folder, 'elevation.npy'))
t_Z = np.load(os.path.join(folder, 'elevation_t.npy'))
Wdat = np.load(os.path.join(folder, 'width.npy'))
t_W = np.load(os.path.join(folder, 'width_t.npy'))
x_W = (np.arange(Wdat.shape[1])+0.5)/10

Zsum = np.nansum(Z, axis=1)
Vsand = np.cumsum(Zsum, axis=1) * 0.65 * 1e-6

#skip doubled scans
ixdrop = np.argwhere(np.diff(t_Z)[1:] - (np.diff(t_Z[:-1]) - 100) < 0).ravel()+1
ixkeep = np.delete(np.arange(len(t_Z)), ixdrop)

t_Z = t_Z[ixkeep]
Vsand = Vsand[ixkeep, :]

# skip additional scans
#ixkeep = np.array([0, 1, 2, 4, 5, 7, 9, 12, 15, 20, 25, 32, 41, 49, 69, 77, 87])
#t_Z2 = t_Z1[ixkeep]
#Vsand2 = Vsand1[ixkeep, :]

dv = np.diff(Vsand, axis=0)
dt = np.diff(t_Z)
dvdt = np.array([dv[i,:]/dt[i] for i in range(len(dt))])
tdat = (t_Z[:-1] * t_Z[1:])**(1/2)

p = 0.35
Qdat = (1/(1-p)) * -dvdt

NameError: name 'Z' is not defined

In [209]:
xres = 10 # cm
xbnds = np.arange((fluxdat.shape[1]//xres)+1) * xres
x = xbnds[1:]

xin_W, tin_W = np.meshgrid(x_W, t_W)
x, t = np.meshgrid(x, tdat)

In [199]:
Qx = np.empty(tout.shape)
W = np.empty(tout.shape)
for i in range(tout.shape[0]):
    for j in range(tout.shape[1]):
        Qx[i,j] = Qdat[i, j*xres]
        W[i,j] = np.nanmean(Wdat[(tin_W>t_Z[i]) & (tin_W<t_Z[i+1]) & (xin_W>xbnds[j]) & (xin_W<xbnds[j+1])])


x = xout[]
Qx = Qx[1:, 1:]
Qx = Qx[1:, 1:]


qx = Qx/W

C:\Users\tashl\AppData\Local\Temp/ipykernel_22196/3233548877.py:6: RuntimeWarning: Mean of empty slice
  W[i,j] = np.nanmean(Wdat[(tin_W>t_Z[i]) & (tin_W<t_Z[i+1]) & (xin_W>xbnds[j]) & (xin_W<xbnds[j+1])])
